# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2025-01-05T21:05:38.609-0600	connected to: mongodb://localhost/
2025-01-05T21:05:38.610-0600	dropping: uk_food.establishments
2025-01-05T21:05:39.432-0600	39779 document(s) imported successfully. 0 document(s) failed to import.


In [7]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [12]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'St Margarets Hotel And Country Club',
 'AddressLine2': 'Reach Road',
 'AddressLine3': 'St Margarets At Cliffe',
 'AddressLine4': 'Kent',
 'BusinessName': 'Boat House & Langdon',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4588.067654106012,
 'FHRSID': 254239,
 'LocalAuthorityBusinessID': 'PI/000068711',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6AG',
 'RatingDate': '2021-07-01T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677b4882650f6cc83d949b58'),
 'geocode': {'latitude': '51.150372', 'longitude': '1.366199'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254239',
            'rel': 'self'}],
 'meta': {'dataSource

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_establishment = {'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen'}

In [14]:
# Insert the new restaurant into the collection
establishments.insert_one(new_establishment)

InsertOneResult(ObjectId('677b4a1590d8c60dfb137108'), acknowledged=True)

In [15]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = list(establishments.find(query))
results

[{'_id': ObjectId('677b4a1590d8c60dfb137108'),
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen'}]

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [20]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1}
results = list(establishments.find(query, fields).limit(1))
results[0]

{'_id': ObjectId('677b4882650f6cc83d949b58'), 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [24]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName': 'Penang Flavours'},{'$set':{'BusinessTypeID': 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [25]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = list(establishments.find(query))
results

[{'_id': ObjectId('677b4a1590d8c60dfb137108'),
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1}]

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [29]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
results = list(establishments.find(query))
len(results)

994

In [30]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [31]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
results = list(establishments.find(query))
len(results)

0

In [32]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677b4882650f6cc83d949e2e'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [35]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set': { "latitude" : {'$toDouble': "$latitude"}} } ])
establishments.update_many({}, [ {'$set': { "longitude" : {'$toDouble': "$longitude"}} } ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [36]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [37]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set': { "RatingValue" : {'$toInt': "$RatingValue"}} } ])

UpdateResult({'n': 38786, 'nModified': 34644, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [38]:
# Check that the coordinates and rating value are now numbers
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677b4882650f6cc83d949e2e'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'longitude': None,
 'me